q2text

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/qwen72b")

In [2]:
from datasets import load_from_disk

question1_set = load_from_disk("data/instructions/gen_ins/ins1")
question2_set = load_from_disk("data/instructions/gen_ins/ins2")
question1_set, question2_set

(Dataset({
     features: ['question'],
     num_rows: 209971
 }),
 Dataset({
     features: ['question'],
     num_rows: 209971
 }))

In [3]:
question1_set[0], question2_set[0]

({'question': 'What factors contribute to the low efficiency of cationic lipid-mediated gene transduction in polarized and differentiated airway epithelial cells, and how might these barriers be overcome to improve gene therapy for airway diseases such as cystic fibrosis?'},
 {'question': 'How do the results of gene transduction using GL-67:pDNA complexes in the nasal epithelium of cystic fibrosis transgenic mice compare to those observed in the lungs, and what implications do these differences have for the development of gene therapy strategies for cystic fibrosis?'})

In [4]:
p = '''
You need to evaluate the given query based on the following criteria and output the results in JSON format. The output should include three parts: quality, difficulty, and whether additional necessary information is required to answer the query. Please follow the scoring standards below:
1. Quality (Score 1-10): Assess the clarity and accuracy of the query. If the query is a simple statement without any question or instruction, score it 1-2.
    9-10: Very clear, accurate expression, no ambiguity.
    7-8: Clear, accurate expression, but may have minimal ambiguity.
    5-6: Fairly clear, generally accurate expression, but some ambiguity exists.
    3-4: Not very clear, somewhat vague expression, with obvious ambiguity.
    1-2: Unclear, very vague expression, difficult to understand or a simple statement.
2. Difficulty (Score 1-10): Assess the difficulty of understanding and answering the query.
    9-10: Very difficult, requires specialized knowledge and complex analysis to answer.
    7-8: Quite difficult, requires some specialized knowledge and analysis.
    5-6: Moderate difficulty, requires general knowledge and analysis.
    3-4: Fairly simple, can be answered with basic knowledge.
    1-2: Very simple, no special knowledge required to answer.
3. Relevance to medicine (1-6): Assess the medical relevance of the query.
    5-6: Completely related to medicine, with many medical terms appearing.
    3-4: Related to medicine, the content involves medical fields.
    1-2: Very weak medical relevance.
4. Mention specific details: Whether specific case details in the text are mentioned.
Please only output the results without any explanation and strictly follow the format below for output:
{
  "quality": 1-10,
  "difficulty": 1-10,
  "relevance_to_medicine": 1-6,
  "mention_specific_details": "True"/"False"
}
Here is the query:
'''
p

'\nYou need to evaluate the given query based on the following criteria and output the results in JSON format. The output should include three parts: quality, difficulty, and whether additional necessary information is required to answer the query. Please follow the scoring standards below:\n1. Quality (Score 1-10): Assess the clarity and accuracy of the query. If the query is a simple statement without any question or instruction, score it 1-2.\n    9-10: Very clear, accurate expression, no ambiguity.\n    7-8: Clear, accurate expression, but may have minimal ambiguity.\n    5-6: Fairly clear, generally accurate expression, but some ambiguity exists.\n    3-4: Not very clear, somewhat vague expression, with obvious ambiguity.\n    1-2: Unclear, very vague expression, difficult to understand or a simple statement.\n2. Difficulty (Score 1-10): Assess the difficulty of understanding and answering the query.\n    9-10: Very difficult, requires specialized knowledge and complex analysis to

In [7]:
def raw2text(example):
    text_parts = ['You need to evaluate the given query based on the following criteria and output the results in JSON format. The output should include three parts: quality, difficulty, and whether additional necessary information is required to answer the query. Please follow the scoring standards below:\n1. Quality (Score 1-10): Assess the clarity and accuracy of the query. If the query is a simple statement without any question or instruction, score it 1-2.\n    9-10: Very clear, accurate expression, no ambiguity.\n    7-8: Clear, accurate expression, but may have minimal ambiguity.\n    5-6: Fairly clear, generally accurate expression, but some ambiguity exists.\n    3-4: Not very clear, somewhat vague expression, with obvious ambiguity.\n    1-2: Unclear, very vague expression, difficult to understand or a simple statement.\n2. Difficulty (Score 1-10): Assess the difficulty of understanding and answering the query.\n    9-10: Very difficult, requires specialized knowledge and complex analysis to answer.\n    7-8: Quite difficult, requires some specialized knowledge and analysis.\n    5-6: Moderate difficulty, requires general knowledge and analysis.\n    3-4: Fairly simple, can be answered with basic knowledge.\n    1-2: Very simple, no special knowledge required to answer.\n3. Relevance to medicine (1-6): Assess the medical relevance of the query.\n    5-6: Completely related to medicine, with many medical terms appearing.\n    3-4: Related to medicine, the content involves medical fields.\n    1-2: Very weak medical relevance.\n4. Mention specific details: Whether specific case details in the text are mentioned.\nPlease only output the results without any explanation and strictly follow the format below for output:\n{\n  "quality": 1-10,\n  "difficulty": 1-10,\n  "relevance_to_medicine": 1-6,\n  "mention_specific_details": "True"/"False"\n}\nHere is the query:\n']
    text_parts.append(example["question"])
    
    text = "".join(text_parts)
    text += "\nOutput: \n" 
    example["text"] = text
    return example

In [8]:
question1_set = question1_set.map(raw2text, remove_columns=question1_set.column_names)
question2_set = question2_set.map(raw2text, remove_columns=question2_set.column_names)
question1_set, question2_set

Map:   0%|          | 0/209971 [00:00<?, ? examples/s]

Map: 100%|██████████| 209971/209971 [00:06<00:00, 31582.46 examples/s]


(Dataset({
     features: ['text'],
     num_rows: 209971
 }),
 Dataset({
     features: ['text'],
     num_rows: 209971
 }))

In [9]:
question1_set[0], question2_set[0]

({'text': 'You need to evaluate the given query based on the following criteria and output the results in JSON format. The output should include three parts: quality, difficulty, and whether additional necessary information is required to answer the query. Please follow the scoring standards below:\n1. Quality (Score 1-10): Assess the clarity and accuracy of the query. If the query is a simple statement without any question or instruction, score it 1-2.\n    9-10: Very clear, accurate expression, no ambiguity.\n    7-8: Clear, accurate expression, but may have minimal ambiguity.\n    5-6: Fairly clear, generally accurate expression, but some ambiguity exists.\n    3-4: Not very clear, somewhat vague expression, with obvious ambiguity.\n    1-2: Unclear, very vague expression, difficult to understand or a simple statement.\n2. Difficulty (Score 1-10): Assess the difficulty of understanding and answering the query.\n    9-10: Very difficult, requires specialized knowledge and complex ana

text2prompt

In [10]:
messages_template = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "{prompt}"}
]

def create_messages(prompt):
    return [
        msg.copy() if msg["role"] == "system" 
        else {"role": "user", "content": prompt}
        for msg in messages_template
    ]

In [11]:
def create_prompt(example):
    message = create_messages(example["text"])
    text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True
    )
    example["prompt"] = text
    return example

In [12]:
question1_set = question1_set.map(create_prompt, remove_columns=["text"])
question2_set = question2_set.map(create_prompt, remove_columns=["text"])
question1_set, question2_set

Map: 100%|██████████| 209971/209971 [00:17<00:00, 11903.07 examples/s]


(Dataset({
     features: ['prompt'],
     num_rows: 209971
 }),
 Dataset({
     features: ['prompt'],
     num_rows: 209971
 }))

In [13]:
question1_set[0], question2_set[0]

({'prompt': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nYou need to evaluate the given query based on the following criteria and output the results in JSON format. The output should include three parts: quality, difficulty, and whether additional necessary information is required to answer the query. Please follow the scoring standards below:\n1. Quality (Score 1-10): Assess the clarity and accuracy of the query. If the query is a simple statement without any question or instruction, score it 1-2.\n    9-10: Very clear, accurate expression, no ambiguity.\n    7-8: Clear, accurate expression, but may have minimal ambiguity.\n    5-6: Fairly clear, generally accurate expression, but some ambiguity exists.\n    3-4: Not very clear, somewhat vague expression, with obvious ambiguity.\n    1-2: Unclear, very vague expression, difficult to understand or a simple statement.\n2. Difficulty (Score 1-10): Assess the difficulty of understanding and answering the 

In [14]:
question1_set.save_to_disk("data/scoring/ins1_prompt")
question2_set.save_to_disk("data/scoring/ins2_prompt")

Saving the dataset (1/1 shards): 100%|██████████| 209971/209971 [00:00<00:00, 389323.73 examples/s]
